Tests for the file written in pythpn

In [2]:
import ifcopenshell
from ifcopenshell.api import run
import ifcopenshell.util.element
import ifcopenshell.util.selector

In [4]:
model = ifcopenshell.open("../project_data/script_output_4x3.ifc")

In [20]:
def Modellaufbau_Borehole_GeotechnicalStratum(model):
    """
    Beschreibung:
        Ein IfcBorehole dient als Container für verschiedene IfcGeotechnicalStratum. IfcBorehole beschreibt die Bohrung als ganzes, wohin IfcGeotechnicalStratum zur Repräsentation von Ansprachebereichen verwendet wird.
    Anweisung:
        Der Modellaufbau ist durch den Fachplaner für Geotechnik sicherzustellen.
    """

    # Filtern der Elemente
    elems = model.by_type("IfcGeotechnicalStratum")
    elems = [i for i in elems if i.ObjectType == "ANSPRACHEBEREICH"]
    
    # Alternativ unter Nutzung der Query Syntax
    elems = ifcopenshell.util.selector.filter_elements(model, "IfcGeotechnicalStratum, ObjectType=ANSPRACHEBEREICH")

    # Für jedes Element: Prüfen der Anforderung
    passed, failed = [], []
    for elem in elems:
        if not elem.Decomposes:
            failed.append(elem)
            continue
        if any((i.RelatingObject.is_a("IfcBorehole") and i.is_a("IfcRelAggregates")) for i in elem.Decomposes): # Hinweis: Hier muss minndestens eine der IfcRelAggregates auf ein IfcBorehole verweisen. Wenn es genau eins sein soll, dann len(elem.Decomposes) == 1 und elem.Decomposes[0] testen
            passed.append(elem)
        else:
            failed.append(elem)
    
    print(f"{len(passed)} Elemente haben bestanden, {len(failed)} sind durchgefallen.")

Modellaufbau_Borehole_GeotechnicalStratum(model)    


15 Elemente haben bestanden, 0 sind durchgefallen.


In [ ]:
def WerteBereich_Kohaesion(model, min_val=0, max_val=1_000):
    """
    Prüft, ob die Eigenschaft CohesionBehaviour im Pset_SolidStratumCapacity in einem Wertebereich liegt
    """
    # Filtern der Elemente
    # Hinweis: IfcSimpleProperties können auf mehrere Arten beschrieben werden, die am häufigst verwendete ist IfcPropertySingleValue
    elems = model.by_type("IfcSimpleProperty")
    #elems = model.by_type("IfcPropertySingleValue") + model.by_type("IfcPropertyBoundedValue") + model.by_type("IfcPropertyEnumeratedValue") + model.by_type("IfcPropertyListValue") + model.by_type("IfcPropertyReferenceValue") + model.by_type("IfcPropertyTableValue")

    elems = [i for i in elems if i.Name=="CohesionBehaviour"]
    elems = [i for i in elems if any(j.Name=="Pset_SolidStratumCapacity" for j in i.PartOfPset)]

    # Für jedes Element: Prüfen der Anforderung
    passed, failed = [], []
    for elem in elems:
        value = elem.NominalValue.wrappedValue
        if value>=min_val and value<=max_val: # Hinweis: Im IDS Beispiel werden die beiden Tests gesondert gelistet.
            passed.append(elem)
        else:
            failed.append(elem)

    #    print(elem.NominalValue, elem.Unit)
    #    print(elem.HasConstraints)
    #print(dir(elem.NominalValue))
    #print(elem.NominalValue.wrappedValue)


WerteBereich_Kohaesion(model)


IfcPressureMeasure(0.) None
()
IfcPressureMeasure(5.) None
()
IfcPressureMeasure(10000.) None
()
['__annotations__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rge__', '__rgt__', '__rle__', '__rlt__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'attribute_name', 'attribute_type', 'compare', 'file', 'get_info', 'get_info_2', 'id', 'is_a', 'is_entity', 'to_string', 'unwrap_value', 'walk', 'wrap_value', 'wrappedValue']
10000.0
